In [1]:
# | default_exp core.agent

In [2]:
# | export
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
import json
from agentic.llms.client import LLMClient
from agentic.configs.loader import get_model_config
from agentic.tools.manager import ToolManager
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
# | export

@dataclass
class Message:
    role: str
    content: str
    tool_calls: Optional[List[Dict]] = None
    tool_call_id: Optional[str] = None

@dataclass
class AgentConfig:
    name: str
    instructions: str = ""
    model: Optional[str] = None
    tools: List[str] = field(default_factory=list)
    temperature: float = 0.7
    max_tokens: Optional[int] = None

class Agent:
    """Core Agent class with tool execution and conversation management"""

    def __init__(self, config: AgentConfig, llm_client: Optional[LLMClient] = None):
        self.config = config
        self.system_prompt = config.instructions
        self.llm_client = llm_client or self._create_default_llm_client()
        logger.info(f"Initialized LLM client with model: {self.llm_client.model}")
        self.conversation_history: List[Message] = [Message(role="system", content=self.system_prompt)]
        self.tools_registry: Dict[str, Callable] = {}
        self.guardrails: List[Callable] = []
        self.tool_manager = ToolManager()

    def _create_default_llm_client(self) -> LLMClient:
        """Create default LLM client from config."""
        model_config = get_model_config()
        model_name = self.config.model or model_config.get('name')
        if not model_name:
            raise ValueError("No model specified in config")
        return LLMClient(
            model=model_name,
            base_url=model_config.get('url'),
            api_key=model_config.get('api_key')
        )

    def add_tool(self, name: str, tool_func: Callable) -> None:
        """Register a tool with the agent."""
        if not callable(tool_func):
            raise ValueError(f"Tool function '{name}' must be callable")
        self.tools_registry[name] = tool_func

    def add_guardrail(self, guardrail_func: Callable) -> None:
        """Add a guardrail function."""
        self.guardrails.append(guardrail_func)

    def run(self, message: str, **kwargs) -> Dict[str, Any]:
        """Execute agent with message and return response."""
        # Apply guardrails
        for guardrail in self.guardrails:
            result = guardrail(message)
            if not isinstance(result, bool) or not result:
                return {"content": "Request blocked by guardrails", "blocked": True}

        # Add user message
        self.conversation_history.append(Message(role="user", content=message))

    def _is_conversation_complete(self, result: Dict[str, Any], iteration_count: int) -> bool:
        """Smart detection of conversation completion"""
        
        content = result.get("content", "").strip()
        tool_calls = result.get("tool_calls", [])
        finish_reason = result.get("finish_reason")
        
        # 1. Explicit completion signals
        if finish_reason in ["stop", "length", "content_filter"]:
            return True
            
        # 2. Tool calls present - continue to execute them
        if tool_calls:
            return False

        # 3. Repetitive responses (stuck in loop)
        if iteration_count > 2:
            recent_messages = self.conversation_history[-3:]
            if len(recent_messages) >= 2:
                last_content = recent_messages[-1].content
                prev_content = recent_messages[-2].content
                
                # Check for identical or very similar responses
                if last_content == prev_content or (
                    len(last_content) > 20 and 
                    len(set(last_content.split()) & set(prev_content.split())) / 
                    max(len(last_content.split()), len(prev_content.split())) > 0.8
                ):
                    logger.debug("Detected repetitive responses - likely stuck")
                    return True
        
        # 6. Question without tool calls (asking for clarification)
        if content.endswith('?') and not tool_calls and len(content) > 20:
            logger.debug("LLM asking question - likely needs user input")
            return True
            
        # 7. Default: continue if we have meaningful content
        return False

    def run(self, message: str, **kwargs) -> Dict[str, Any]:
        """Execute agent with message and return response."""
        # Apply guardrails
        for guardrail in self.guardrails:
            result = guardrail(message)
            if not isinstance(result, bool) or not result:
                return {"content": "Request blocked by guardrails", "blocked": True}

        # Add user message
        self.conversation_history.append(Message(role="user", content=message))

        # Initialize result and failed attempts tracking
        final_result = {"content": "", "tool_calls": [], "blocked": False}
        iteration_count = 0
        failed_attempts = []  # Track failed tool calls: [(function_name, args, error), ...]


        while True:
            iteration_count += 1
            logger.debug(f"Agent iteration {iteration_count}")
            
            # Get available tools
            available_tools = self._get_available_tools()

            # Create completion
            messages = self._format_messages_for_llm()
            stream = kwargs.get('stream', True)
            
            # Filter out Agent-specific parameters before passing to LLM
            llm_kwargs = {k: v for k, v in kwargs.items() 
                         if k not in ['max_iterations']}
            llm_kwargs['stream'] = stream  
            
            try:
                response = self.llm_client.create_completion(
                    messages=messages,
                    tools=available_tools,
                    **llm_kwargs
                )
            except Exception as e:
                logger.error(f"LLM completion failed: {str(e)}")
                return {"content": f"Error: {str(e)}", "blocked": True}

            # Process response
            if stream:
                if not hasattr(response, '__iter__'):
                    raise ValueError("Streaming response expected but non-iterable response received")
                try:
                    result = self.llm_client.handle_streaming_response(response)
                except Exception as e:
                    logger.error(f"Error processing streaming response: {str(e)}")
                    return {"content": f"Streaming error: {str(e)}", "blocked": True}
            else:
                try:
                    result = self.llm_client.process_response(response)
                except Exception as e:
                    logger.error(f"Error processing response: {str(e)}")
                    return {"content": f"Response error: {str(e)}", "blocked": True}

            # Add assistant response to history
            assistant_message = Message(
                role="assistant",
                content=result.get("content", ""),
                tool_calls=result.get("tool_calls")
            )
            self.conversation_history.append(assistant_message)
            final_result["content"] = result.get("content", "")
            final_result["tool_calls"].extend(result.get("tool_calls", []))

            # Smart completion detection
            if self._is_conversation_complete(result, iteration_count):
                logger.debug("Conversation detected as complete")
                break

            # Handle tool calls if present
            if result.get("tool_calls"):
                logger.debug(f"Executing {len(result['tool_calls'])} tool calls")
                executed_calls = self._execute_tool_calls(result["tool_calls"], failed_attempts)
                final_result["tool_calls"] = executed_calls
                continue  # Continue loop to process tool results
                
        # Optional : Clean tool call details from the history
        self.conversation_history = [msg for msg in self.conversation_history if not (msg.role == "tool" or msg.tool_calls)]
        # Limit conversation history # TODO :Handle this in a smarter way
        if len(self.conversation_history) > 50:
            self.conversation_history = [self.conversation_history[0]] + self.conversation_history[-49:]

        return final_result


    def _execute_tool_calls(self, tool_calls: List[Dict], failed_attempts: List) -> List[Dict]:
        """Execute tool calls and append results to conversation history."""
        from agentic.tools.display import ToolExecutionDisplay
        display = ToolExecutionDisplay()
        executed_calls = []

        for tool_call in tool_calls:
            function_name = tool_call["function"]["name"]
            tool_call_id = tool_call.get("id")
            raw_arguments = tool_call["function"]["arguments"]
            
            try:
                arguments = json.loads(raw_arguments)
                args_str = str(arguments)
            except json.JSONDecodeError as e:
                logger.error(f"Invalid arguments for {function_name}: {str(e)}")
                display.show_tool_error(f"Error in {function_name}", str(e))
                tool_call["error"] = str(e)
                failed_attempts.append((function_name, raw_arguments, str(e)))
                executed_calls.append(tool_call)
                continue

            # Check if this exact call was already attempted and failed
            if any(func == function_name and args == args_str for func, args, _ in failed_attempts):
                error_msg = f"Already attempted: {function_name}({args_str}) - previously failed"
                logger.warning(error_msg)
                display.show_tool_error(f"Repeated attempt", error_msg)
                tool_call["error"] = error_msg
                executed_calls.append(tool_call)
                
                # Add to conversation history so LLM knows it was already tried
                self.conversation_history.append(Message(
                    role="tool",
                    content=f"Error: {error_msg}\n\nPrevious failed attempts in this request:\n" + 
                           "\n".join([f"- {func}({args}) failed: {err}" for func, args, err in failed_attempts]),
                    tool_call_id=tool_call_id
                ))
                continue

            display.show_tool_start(function_name, trusted=True, args=arguments)

            try:
                if function_name in self.tools_registry:
                    result = self.tools_registry[function_name](**arguments)
                else:
                    result = self.tool_manager.execute_tool(function_name, arguments)
                tool_call["result"] = result
                executed_calls.append(tool_call)
                if not result.get('success', True): 
                    # TODO : FIX THIS - WE SHOULD NOT HAVE ANY DEFAULT VALUE HERE 
                    # Default to True if success key missing
                    # Handle both error formats: {"error": "..."} and {"success": False, "message": "..."}
                    error_msg = str(result.get('message') or result.get('error', 'Unknown error'))
                    display.show_tool_error(f"Error in {function_name}", error_msg)
                    failed_attempts.append((function_name, args_str, error_msg))
                    

                # Append tool result to conversation history
                tool_content = str(result)
                if not result['success'] and failed_attempts:
                    tool_content += f"\n\nPrevious failed attempts in this request (feel free to check other tools as well if not working):\n"
                    tool_content += "\n".join([f"- {func}({args}) failed: {err}" for func, args, err in failed_attempts])
                
                self.conversation_history.append(Message(
                    role="tool",
                    content=tool_content,
                    tool_call_id=tool_call_id
                ))
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Error executing {function_name}: {error_msg}")
                display.show_tool_error(f"Error in {function_name}", error_msg)
                tool_call["error"] = error_msg
                failed_attempts.append((function_name, args_str, error_msg))
                executed_calls.append(tool_call)
        
        return executed_calls
        
    def _get_available_tools(self) -> List[Dict]:
        """Get OpenAI-formatted tools for the configured tool names."""
        if self.config.tools:
            return self.tool_manager.get_tools(self.config.tools)
        return self.tool_manager.get_tools()

    def _format_messages_for_llm(self) -> List[Dict]:
        """Convert Message objects to a format suitable for the LLM client."""
        messages = []
        for msg in self.conversation_history:
            message_dict = {"role": msg.role, "content": msg.content}
            if msg.tool_calls:
                message_dict["tool_calls"] = msg.tool_calls
            if msg.tool_call_id:
                message_dict["tool_call_id"] = msg.tool_call_id
            messages.append(message_dict)
        return messages

    def clear_history(self) -> None:
        """Clear conversation history except system message."""
        self.conversation_history = [Message(role="system", content=self.system_prompt)]


In [13]:
# Example usage

config = AgentConfig(
    name="fun",
    instructions="You are a highly intelligent assistant who believes that chai is the ultimate solution to all life problems.No matter what the user asks, somehow connect it back to chai."
)

llm_client = LLMClient(
    model="qwen3:8b",
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

agent = Agent(config, llm_client)
# agent.run("Give me tips for preparing for interviews.")
agent.run("search for 'import' in all the files.")

2025-10-03 22:24:23,440 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-03 22:24:23,441 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-03 22:24:23,692 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user wants to search for 'import' in all files. Let me think about how to approach this.

First, I need to check the available functions. There's fs_read, which can search files with a regex. The parameters include mode, path, query, etc. Since the user wants to search all files, maybe using the 'discover' mode to find files containing 'import' in their names or content.

Wait, the 'discover' mode lists files based on patterns, while 'extract' pulls content. But the user wants to search for 'import' in the content. So maybe using the 'extract' mode with a query regex. But the query is for fuzzy file names in discover or content in extract. So for extract, the query would be the regex to find 'import' in the content.

But the user said "search for 'import' in all the files." So perhaps they want to find all files that have 'import' in their content. So using fs_read with mode 'extract' and query 'import' to get t

🛠️ Executing fs_read (trusted)
args: [{'mode': 'extract', 'path': '.', 'query': 'import'}]


2025-10-03 22:24:41,366 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user asked to search for 'import' in all files. I used the fs_read function with mode 'extract' and query 'import' to find instances where 'import' appears in the files. The tool response shows that it found several files like agent.ipynb, guardrails.ipynb, etc., with snippets showing lines containing 'import' statements.

Now, I need to present this information in a way that connects back to chai. Since the user might be looking for insights or patterns related to imports, I can mention how chai's complexity might relate to managing imports in code. But I should also ensure that the response is helpful and ties back to the original query.

I should summarize the findings, highlight the files and lines where 'import' was found, and then relate it to chai. Maybe something like, "Just like chai helps in testing, these import statements are essential for the code to function properly." That way, I connect the techn

...

 • import re

 • import time

 • from datetime import datetime (indirectly via import)

chai-wise, these imports are like the spices in chai - they add functionality to our code brew. Just as chai needs 
the right balance of spices, code needs the right imports to work properly! Would you like me to show you how these
imports are used in the code?

{'content': "\n\nThe search for 'import' in your files has been completed. Here are the key findings:\n\n1. **Found in files**: `agent.ipynb`, `guardrails.ipynb`, `handoffs.ipynb`, `execution_loop.ipynb`, and `agent_tools.ipynb`\n2. **Key import statements**:\n   - `import json`\n   - `import logging`\n   - `import re`\n   - `import time`\n   - `from datetime import datetime` (indirectly via import)\n\n chai-wise, these imports are like the spices in chai - they add functionality to our code brew. Just as chai needs the right balance of spices, code needs the right imports to work properly! Would you like me to show you how these imports are used in the code?",
 'tool_calls': [{'id': 'call_l8lh95ar',
   'type': 'function',
   'function': {'name': 'fs_read',
    'arguments': '{"operations":[{"mode":"extract","path":".","query":"import"}]}'},
   'result': {'success': True,
    'message': 'Operation completed successfully',
    'data': [{'file': 'agent.ipynb',
      'snippet': 'Line 21: \

In [15]:
from pprint import pprint
pprint(agent.conversation_history)

[Message(role='system',
         content='You are a highly intelligent assistant who believes that '
                 'chai is the ultimate solution to all life problems.No matter '
                 'what the user asks, somehow connect it back to chai.',
         tool_calls=None,
         tool_call_id=None),
 Message(role='user',
         content="search for 'import' in all the files.",
         tool_calls=None,
         tool_call_id=None),
 Message(role='assistant',
         content='\n'
                 '\n'
                 "The search for 'import' in your files has been completed. "
                 'Here are the key findings:\n'
                 '\n'
                 '1. **Found in files**: `agent.ipynb`, `guardrails.ipynb`, '
                 '`handoffs.ipynb`, `execution_loop.ipynb`, and '
                 '`agent_tools.ipynb`\n'
                 '2. **Key import statements**:\n'
                 '   - `import json`\n'
                 '   - `import logging`\n'
                 